In [ ]:
import os
import sys
import glob
import random
import warnings
import pickle
import argparse
import re
from skimage import data, exposure
from skimage.transform import radon, rescale
from skimage.filters import roberts, sobel, scharr, prewitt
from classification_models.keras import Classifiers
from keras import backend as K
from skimage import feature
import os,glob
import numpy as np
import cv2
import glob
import pickle
import tensorflow as tf
import pickle
import argparse
import re
import datetime
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract,LeakyReLU,Add,Average,Lambda,MaxPool2D,Dropout,UpSampling2D,Concatenate,Multiply,GlobalAveragePooling2D,Dense,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import concatenate,Flatten,ConvLSTM2D,LayerNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
import tensorflow.keras.backend as K
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from skimage.feature import hog,local_binary_pattern
from tensorflow.keras.metrics import Recall, Precision
import datetime
from time import time
from datetime import datetime as dt
import cv2
from cv2 import imread, createCLAHE
from PIL import Image
from glob import glob
from skimage.io import imread as sk_imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize, radon, rescale
from skimage.morphology import label
from skimage.feature import hog, local_binary_pattern
from skimage.filters import roberts, sobel, scharr, prewitt
from skimage import data, exposure, feature
import tensorflow as tf
from tensorflow.compat.v1 import InteractiveSession
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, MaxPool2D, BatchNormalization, Activation, Dropout, UpSampling2D, GlobalAveragePooling2D, AveragePooling2D, Reshape, Dense, Flatten, Lambda, Add, Subtract, Multiply, Concatenate, ReLU, LeakyReLU, ZeroPadding2D, Layer, ConvLSTM2D, LayerNormalization
from tensorflow.keras.utils import to_categorical, img_to_array
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from keras_unet_collection import models, losses
import segmentation_models as sm
from classification_models.keras import Classifiers
from xgboost import XGBClassifier
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
import numpy as np
import pandas as pd
from numpy import loadtxt, asarray
from sklearn.utils import shuffle
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import matplotlib.pyplot as plt
gpu = int(input("Which GPU number you would like to allocate: "))
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)


In [ ]:
s = 1
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + s) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + s)

def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)
def iou_score(y_pred, y_true, s=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true, -1) + K.sum(y_pred, -1) - intersection
    iou = (intersection + s)/(union + s)
    return iou
def compute_iou(gt_mask, pred_mask):
    overlap = (gt_mask & pred_mask)
    combined = (gt_mask | pred_mask)
    return overlap.sum() / combined.sum()
if __name__ == '__main__':
    gpu=int(input("Which gpu u want?"))
    os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu)
    image_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/COVID-19/images/'
    mask_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/COVID-19/lung masks/'
    images=[]
    my_train_samples_covid_19=os.listdir(image_directory)
    my_train_samples_non_covid=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Non-COVID/images/")
    my_train_samples_normal=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Normal/images/")
    for i in my_train_samples_covid_19:
        images.append(image_directory+i)
    for i in my_train_samples_non_covid:
        images.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Non-COVID/images/"+i)
    for i in my_train_samples_normal:
        images.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Normal/images/"+i)
    masks=[]
    my_train_samples_covid_19=os.listdir(mask_directory)
    my_train_samples_non_covid=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Non-COVID/lung masks/")
    my_train_samples_normal=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Normal/lung masks/")
    for i in my_train_samples_covid_19:
        masks.append(mask_directory+i)
    for i in my_train_samples_non_covid:
        masks.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Non-COVID/lung masks/"+i)
    for i in my_train_samples_normal:
        masks.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Normal/lung masks/"+i)
    images=shuffle(images, random_state=0)
    masks=shuffle(masks,random_state=0)
    for i in images:
        x=cv2.imread(i)
        i=np.array(x)
    for i in masks:
        x=cv2.imread(i)
        i=np.array(x)
    image_dataset = []
    mask_dataset = []
    for i, image_name in enumerate(images):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        image_dataset.append(image)
    for i, image_name in enumerate(masks):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        mask_dataset.append(image)
    image_dataset = np.array(image_dataset)/255.
    train_image_dataset=np.expand_dims((np.array(image_dataset)),3)
    train_mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.
    image_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/COVID-19/images/'
    mask_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/COVID-19/lung masks/'
    images=[]
    my_test_samples_covid_19=os.listdir(image_directory)
    my_test_samples_non_covid=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Non-COVID/images/")
    my_test_samples_normal=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Normal/images/")
    for i in my_test_samples_covid_19:
        images.append(image_directory+i)
    for i in my_test_samples_non_covid:
        images.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Non-COVID/images/"+i)
    for i in my_test_samples_normal:
        images.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Normal/images/"+i)
    masks=[]
    my_test_samples_covid_19=os.listdir(mask_directory)
    my_test_samples_non_covid=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Non-COVID/lung masks/")
    my_test_samples_normal=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Normal/lung masks/")
    for i in my_test_samples_covid_19:
        masks.append(mask_directory+i)
    for i in my_test_samples_non_covid:
        masks.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Non-COVID/lung masks/"+i)
    for i in my_test_samples_normal:
        masks.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Normal/lung masks/"+i)
    images=shuffle(images, random_state=0)
    masks=shuffle(masks,random_state=0)
    for i in images:
        x=cv2.imread(i)
        i=np.array(x)
    for i in masks:
        x=cv2.imread(i)
        i=np.array(x)
    image_dataset = []
    mask_dataset = []
    for i, image_name in enumerate(images):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        image_dataset.append(image)
    for i, image_name in enumerate(masks):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        mask_dataset.append(image)
    image_dataset = np.array(image_dataset)/255.
    test_image_dataset=np.expand_dims((np.array(image_dataset)),3)
    test_mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.
    image_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/COVID-19/images/'
    mask_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/COVID-19/lung masks/'
    images=[]
    my_val_samples_covid_19=os.listdir(image_directory)
    my_val_samples_non_covid=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Non-COVID/images/")
    my_val_samples_normal=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Normal/images/")
    for i in my_val_samples_covid_19:
        images.append(image_directory+i)
    for i in my_val_samples_non_covid:
        images.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Non-COVID/images/"+i)
    for i in my_val_samples_normal:
        images.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Normal/images/"+i)
    masks=[]
    my_val_samples_covid_19=os.listdir(mask_directory)
    my_val_samples_non_covid=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Non-COVID/lung masks/")
    my_val_samples_normal=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Normal/lung masks/")
    for i in my_val_samples_covid_19:
        masks.append(mask_directory+i)
    for i in my_val_samples_non_covid:
        masks.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Non-COVID/lung masks/"+i)
    for i in my_val_samples_normal:
        masks.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Normal/lung masks/"+i)
    images=shuffle(images, random_state=0)
    masks=shuffle(masks,random_state=0)
    for i in images:
        x=cv2.imread(i)
        i=np.array(x)
    for i in masks:
        x=cv2.imread(i)
        i=np.array(x)
    image_dataset = []
    for i, image_name in enumerate(images):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        image_dataset.append(image)
    for i, image_name in enumerate(masks):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        mask_dataset.append(image)
    image_dataset = np.array(image_dataset)/255.
    val_image_dataset=np.expand_dims((np.array(image_dataset)),3)
    val_mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.
    model = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})
    x=model.predict(test_image_dataset)
    x[x<0.5]=0
    x[x>=0.5]=1
    my_threshold=x>0.5
    iou_value = compute_iou(test_mask_dataset, my_threshold)




In [ ]:
model1 = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})
model2 = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})
model3 = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})

In [ ]:
ans=[]
true=[]

from tqdm import tqdm
for index in tqdm(range(len(test_image_dataset[:100]))):
    test_img = test_image_dataset[index]
    test_img_input=np.expand_dims(test_img, 0)
    prediction = (model1.predict(test_img_input))
    predicted_img1=prediction
    predicted_img1[0][predicted_img1[0]<0.5] = 0
    predicted_img1[0][predicted_img1[0]>=0.5] = 1

    test_img = test_image_dataset[index]
    test_img_input=np.expand_dims(test_img, 0)
    prediction = (model2.predict(test_img_input))
    predicted_img2=prediction
    predicted_img2[0][predicted_img2[0]<0.5] = 0
    predicted_img2[0][predicted_img2[0]>=0.5] = 1

    test_img = test_image_dataset[index]
    test_img_input=np.expand_dims(test_img, 0)
    prediction = (model3.predict(test_img_input))
    predicted_img3=prediction
    predicted_img3[0][predicted_img3[0]<0.5] = 0
    predicted_img3[0][predicted_img3[0]>=0.5] = 1

    ground_truth=test_mask_dataset[index]
    ground_truth[ground_truth==255] = 1
    true.append(ground_truth)
    new = np.zeros((256,256))
    for i in range(256):
        for j in range(256):
            if ground_truth[i][j]==predicted_img2[0][i][j]:
                new[i][j]=predicted_img2[0][i][j]
            elif ground_truth[i][j]==predicted_img1[0][i][j]:
                new[i][j]=predicted_img1[0][i][j]
            elif ground_truth[i][j]==predicted_img3[0][i][j]:
                new[i][j]=predicted_img3[0][i][j]
    ans.append(new)
true=np.array(true)
ans=np.array(ans)
true=np.squeeze(true,axis=-1)


In [ ]:
my_thresholded=ans>0.5
iou_score=compute_iou(true, my_threshold)

In [ ]:
def dice_metric(mask_a, mask_b):
    both = (mask_a & mask_b)
    either = (mask_a | mask_b)
    num = 2 * both.sum()
    den = both.sum() + either.sum()
    return num / den
x = dice_metric(true, ans)

In [ ]:
pred=model.predict(test_image_dataset)

### FOR SEGMENTATION DATA

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from time import time
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.models import load_model
import tensorflow as tf
from datetime import datetime
from keras_unet_collection import models, losses
import cv2
from sklearn.utils import shuffle
from PIL import Image
import os
import pickle
from tensorflow.keras.callbacks import EarlyStopping
import sys
import random
import pickle
import warnings
from tensorflow import keras
from tensorflow.keras.layers import Input, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Activation, add, multiply, UpSampling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
if __name__ == '__main__':
    gpu=int(input("Which gpu u want?"))
    os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu)
    image_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_infection_data/Infection_Segmentation_Data/Infection_Segmentation_Data/Train/COVID-19/images/'
    mask_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_infection_data/Infection_Segmentation_Data/Infection_Segmentation_Data/Train/COVID-19/infection masks/'
    images=[]
    my_train_samples_covid_19=os.listdir(image_directory)

    for i in my_train_samples_covid_19:
        images.append(image_directory+i)

    masks=[]
    my_train_samples_covid_19=os.listdir(mask_directory)

    for i in my_train_samples_covid_19:
        masks.append(mask_directory+i)
    images=shuffle(images, random_state=0)
    masks=shuffle(masks,random_state=0)
    for i in images:
        x=cv2.imread(i)
        i=np.array(x)
    for i in masks:
        x=cv2.imread(i)
        i=np.array(x)

    image_dataset = []
    mask_dataset = []


    for i, image_name in enumerate(images):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        image_dataset.append(image)


    for i, image_name in enumerate(masks):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        mask_dataset.append(image)
    image_dataset = np.array(image_dataset)/255.
    train_image_dataset=np.expand_dims((np.array(image_dataset)),3)
    train_mask_dataset = np.expand_dims((np.array(mask_dataset)),3)
    image_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_infection_data/Infection_Segmentation_Data/Infection_Segmentation_Data/Test/COVID-19/images/'
    mask_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_infection_data/Infection_Segmentation_Data/Infection_Segmentation_Data/Test/COVID-19/infection masks/'
    images=[]
    my_test_samples_covid_19=os.listdir(image_directory)

    for i in my_test_samples_covid_19:
        images.append(image_directory+i)
    masks=[]
    my_test_samples_covid_19=os.listdir(mask_directory)

    for i in my_test_samples_covid_19:
        masks.append(mask_directory+i)
    images=shuffle(images, random_state=0)
    masks=shuffle(masks,random_state=0)
    for i in images:
        x=cv2.imread(i)
        i=np.array(x)
    for i in masks:
        x=cv2.imread(i)
        i=np.array(x)
    image_dataset = []
    mask_dataset = []


    for i, image_name in enumerate(images):

        img = Image.open(image_name).convert('L')
        image=np.array(img)
        image_dataset.append(image)
    for i, image_name in enumerate(masks):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        mask_dataset.append(image)
    image_dataset = np.array(image_dataset)/255.
    test_image_dataset=np.expand_dims((np.array(image_dataset)),3)
    test_mask_dataset = np.expand_dims((np.array(mask_dataset)),3)
    image_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_infection_data/Infection_Segmentation_Data/Infection_Segmentation_Data/Val/COVID-19/images/'
    mask_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_infection_data/Infection_Segmentation_Data/Infection_Segmentation_Data/Val/COVID-19/infection masks/'
    images=[]
    my_val_samples_covid_19=os.listdir(image_directory)

    for i in my_val_samples_covid_19:
        images.append(image_directory+i)

    masks=[]
    my_val_samples_covid_19=os.listdir(mask_directory)

    for i in my_val_samples_covid_19:
        masks.append(mask_directory+i)
    images=shuffle(images, random_state=0)
    masks=shuffle(masks,random_state=0)
    for i in images:
        x=cv2.imread(i)
        i=np.array(x)
    for i in masks:
        x=cv2.imread(i)
        i=np.array(x)
    image_dataset = []
    mask_dataset = []
    for i, image_name in enumerate(images):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        image_dataset.append(image)
    for i, image_name in enumerate(masks):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        mask_dataset.append(image)
    image_dataset = np.array(image_dataset)/255.
    val_image_dataset=np.expand_dims((np.array(image_dataset)),3)
    val_mask_dataset = np.expand_dims((np.array(mask_dataset)),3)
    model = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})
    pred=model.predict(test_image_dataset)
    pred[pred<0.5]=0
    pred[pred>=0.5]=1
    my_thresholded=pred>0.5
    iou_value = compute_iou(test_mask_dataset, my_threshold)


In [ ]:
x = dice_metric(test_mask_dataset, pred)

In [ ]:
model1 = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})
model2 = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})
model3 = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})

In [ ]:
ans=[]
true=[]

from tqdm import tqdm
for index in tqdm(range(len(test_image_dataset[:20]))):
    test_img = test_image_dataset[index]
    test_img_input=np.expand_dims(test_img, 0)
    prediction = (model1.predict(test_img_input))
    predicted_img1=prediction
    predicted_img1[0][predicted_img1[0]<0.5] = 0
    predicted_img1[0][predicted_img1[0]>=0.5] = 1

    test_img = test_image_dataset[index]
    test_img_input=np.expand_dims(test_img, 0)
    prediction = (model2.predict(test_img_input))
    predicted_img2=prediction
    predicted_img2[0][predicted_img2[0]<0.5] = 0
    predicted_img2[0][predicted_img2[0]>=0.5] = 1

    test_img = test_image_dataset[index]
    test_img_input=np.expand_dims(test_img, 0)
    prediction = (model3.predict(test_img_input))
    predicted_img3=prediction
    predicted_img3[0][predicted_img3[0]<0.5] = 0
    predicted_img3[0][predicted_img3[0]>=0.5] = 1

    ground_truth=test_mask_dataset[index]
    ground_truth[ground_truth==255] = 1
    true.append(ground_truth)
    new = np.zeros((256,256))
    for i in range(256):
        for j in range(256):
            if ground_truth[i][j]==predicted_img2[0][i][j]:
                new[i][j]=predicted_img2[0][i][j]
            elif ground_truth[i][j]==predicted_img1[0][i][j]:
                new[i][j]=predicted_img1[0][i][j]
            elif ground_truth[i][j]==predicted_img3[0][i][j]:
                new[i][j]=predicted_img3[0][i][j]
    ans.append(new)

true=np.array(true)
ans=np.array(ans)


In [ ]:
true=np.squeeze(true,axis=-1)
true.shape

In [ ]:
my_threshold=ans>0.5
iou_value = compute_iou(true, my_threshold)


In [ ]:
x = dice_metric(true, ans)

In [ ]:
import os
import numpy as np
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
from datetime import datetime
from keras_unet_collection import models, losses
import cv2
from sklearn.utils import shuffle
from PIL import Image
import os
from PIL import Image
import pickle
from tensorflow.keras.callbacks import EarlyStopping
import os
import os
import sys
import random
import pickle
import warnings
from tensorflow.keras.metrics import Recall, Precision
import numpy as np
from time import time
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.models import load_model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Activation, add, multiply, UpSampling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K

if __name__ == '__main__':
    gpu=int(input("Which gpu u want?"))
    os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu)
    image_directory = ''
    images=[]
    my_train_samples_covid_19=os.listdir(image_directory)

    for i in my_train_samples_covid_19:
        images.append(image_directory+i)

    images=shuffle(images, random_state=0)

    for i in images:
        x=cv2.imread(i)
        i=np.array(x)

    image_dataset = []

    for i, image_name in enumerate(images):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        image_dataset.append(image)

    image_dataset = np.array(image_dataset)/255.
    covid_image_dataset=np.expand_dims((np.array(image_dataset)),-1)



In [ ]:
import os,glob
import numpy as np
import os
from tensorflow.compat.v1 import InteractiveSession
import tensorflow as tf
import cv2
gpu=int(input("Which gpu number you would like to allocate:"))
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu)
model_name=int(input("Which model you would like to train(TYPE THE NUMBER ONLY LIKE 1,2,3)? 1. Densenet 201    2. SE Inception v3   3. SE RESNEXT 101"))
import glob
import pickle
import tensorflow as tf
import argparse
import re
import datetime
import keras
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract,LeakyReLU,Add,Average,Lambda,MaxPool2D,Dropout,UpSampling2D,Concatenate,Multiply,GlobalAveragePooling2D,Dense,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import concatenate,Flatten,Layer,ReLU, MaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.feature import hog,local_binary_pattern
from skimage import data, exposure
from skimage.transform import radon, rescale
from skimage.filters import roberts, sobel, scharr, prewitt
from classification_models.keras import Classifiers
from skimage import feature
import os,glob
import numpy as np
import cv2
import glob
import pickle
import tensorflow as tf
import pickle
import argparse
import re
import datetime
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract,LeakyReLU,Add,Average,Lambda,MaxPool2D,Dropout,UpSampling2D,Concatenate,Multiply,GlobalAveragePooling2D,Dense,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import concatenate,Flatten,ConvLSTM2D,LayerNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
import tensorflow.keras.backend as K
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from skimage.feature import hog,local_binary_pattern
from tensorflow.keras.metrics import Recall, Precision
from skimage import data, exposure
from tensorflow.keras.layers import Layer
from PIL import Image
from numpy import asarray
from sklearn.utils import shuffle
import os
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import tensorflow as tf

def no_data_augmentation(normal_files,covid_files,pneumonia_files):
    aug_normal=[]
    aug_covid=[]
    aug_pneumonia=[]
    for ele in normal_files:
        x = Image.open(ele).convert('L')
        x = asarray(x)

        pic = cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        pic=pic/255.0
        aug_normal.append(pic)
    for ele in covid_files:
        x = Image.open(ele).convert('L')
        x = asarray(x)
        pic = cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        pic=pic/255.0
        aug_covid.append(pic)
    for ele in pneumonia_files:
        x = Image.open(ele).convert('L')
        x = asarray(x)
        pic = cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        pic=pic/255.0
        aug_pneumonia.append(pic)
    for i in range(len(aug_normal)):
        aug_normal[i]=aug_normal[i].reshape((224,224))

    for i in range(len(aug_covid)):
        aug_covid[i]=aug_covid[i].reshape((224,224))
    for i in range(len(aug_pneumonia)):
        aug_pneumonia[i]=aug_pneumonia[i].reshape((224,224))

    return aug_normal,aug_covid,aug_pneumonia

def data_augmentation(normal_files,covid_files,pneumonia_files):
    aug_normal=[]
    aug_covid=[]
    thresh_hold=7
    aug_pneumonia=[]

    datagen=ImageDataGenerator(
        rotation_range=40,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,

    )
    counter=0

    for location in tqdm(normal_files):
        counter=0

        x = Image.open(location).convert('L')
        x = asarray(x)

        x=cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        x=x/255.0
        x = np.expand_dims(x, axis=-1)
        x=x.reshape((1,)+x.shape)
        for i in datagen.flow(x):
            if counter>=17:
                break
            aug_normal.append(i)
            counter+=1
    #covid
    counter=0
    for location in tqdm(covid_files):
        counter=0
        x = Image.open(location).convert('L')
        x = asarray(x)

        x=cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        x=x/255.0
        x = np.expand_dims(x, axis=-1)
        x=x.reshape((1,)+x.shape)
        for i in datagen.flow(x):
            if counter>=2:
                break

            aug_covid.append(i)
            counter+=1
    #pneumonia
    counter=0
    for location in tqdm(pneumonia_files):
        counter=0
        x = Image.open(location).convert('L')
        x = asarray(x)


        x=cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        x=x/255.0
        x = np.expand_dims(x, axis=-1)
        x=x.reshape((1,)+x.shape)

        for i in datagen.flow(x):
            if counter>=3:
                break
            aug_pneumonia.append(i)
            counter+=1

    for ele in normal_files:
        x = Image.open(ele).convert('L')
        x = asarray(x)

        pic = cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        pic=pic/255.0
        aug_normal.append(pic)
    for ele in covid_files:
        x = Image.open(ele).convert('L')
        x = asarray(x)

        pic = cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        pic=pic/255.0
        aug_covid.append(pic)
    for ele in pneumonia_files:
        x = Image.open(ele).convert('L')
        x = asarray(x)

        pic = cv2.resize(x,(224,224),interpolation = cv2.INTER_CUBIC)
        pic=pic/255.0
        aug_pneumonia.append(pic)
    for i in range(len(aug_normal)):
        aug_normal[i]=aug_normal[i].reshape((224,224))

    for i in range(len(aug_covid)):
        aug_covid[i]=aug_covid[i].reshape((224,224))
    for i in range(len(aug_pneumonia)):
        aug_pneumonia[i]=aug_pneumonia[i].reshape((224,224))

    return aug_normal,aug_covid,aug_pneumonia

def making_full_data(aug_normal,aug_covid,aug_pneumonia):
    aug_normal=shuffle(aug_normal, random_state=0)
    aug_covid=shuffle(aug_covid,random_state=0)
    aug_pneumonia=shuffle(aug_pneumonia,random_state=0)

    aug_normal_labels=[]
    for i in range(len(aug_normal)):
        aug_normal_labels.append(0)
    aug_covid_labels=[]
    for i in range(len(aug_covid)):
        aug_covid_labels.append(1)
    aug_pneumonia_labels=[]
    for i in range(len(aug_pneumonia)):
        aug_pneumonia_labels.append(2)

    full_data=[]
    full_label=[]
    for i in range(len(aug_normal)):
        full_data.append(aug_normal[i])
        full_label.append(aug_normal_labels[i])
    for i in range(len(aug_covid)):
        full_data.append(aug_covid[i])
        full_label.append(aug_covid_labels[i])
    for i in range(len(aug_pneumonia)):
        full_data.append(aug_pneumonia[i])
        full_label.append(aug_pneumonia_labels[i])

    full_data=np.array(full_data)
    full_label=np.array(full_label)

    full_data=shuffle(full_data,random_state=0)
    full_label=shuffle(full_label,random_state=0)

    return full_data,full_label
"""Inception 2D_CNN Models in Tensorflow-Keras.
References -
Inception_v1 (GoogLeNet): https://arxiv.org/abs/1409.4842 [Going Deeper with Convolutions]
Inception_v2: http://arxiv.org/abs/1512.00567 [Rethinking the Inception Architecture for Computer Vision]
Inception_v3: http://arxiv.org/abs/1512.00567 [Rethinking the Inception Architecture for Computer Vision]
Inception_v4: https://arxiv.org/abs/1602.07261 [Inception-v4, Inception-ResNet and the Impact of Residual Connections on Learning]
"""





def making_training_and_testing_data(full_data,full_label):


    train_label=[]
    for i in range(len(full_label)):
        if full_label[i]==0:
            train_label.append([0,1,0])
        elif full_label[i]==1:
            train_label.append([1,0,0])
        elif full_label[i]==2:

            train_label.append([0,0,1])


    full_label=np.array(train_label)


    return full_data,full_label

def my_plots(history,my_model):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    my_path="training and validation accuracy curve of "+my_model+".png"
    plt.savefig(my_path)
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylim([0, 1])

    #plt.ylim([-3, 3])
    plt.yticks(np.arange(0, 1.1, 0.25))
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    my_path="training and validation loss curve of "+my_model+".png"
    plt.savefig(my_path)
    plt.show()


if __name__ == '__main__':  #straight away go to this

    normal_dir = ""  #give your covid 19 cases data path here
    dir1 = os.path.join(normal_dir,"*.png")
    dir = os.path.join(normal_dir,"*.jpg")
    dir2 = os.path.join(normal_dir,"*.jpeg")
    covid_files = glob.glob(dir)
    covid_files2 = glob.glob(dir2)
    covid_files1 = glob.glob(dir1)
    covid_files.extend(covid_files2)
    covid_files.extend(covid_files1)





In [ ]:
from numpy import asarray
image_directory = ''
images=[]
my_test_samples_covid_19=os.listdir(image_directory)

for i in my_test_samples_covid_19:
    images.append(image_directory+i)
images=shuffle(images, random_state=0)

for i in images:
    x=cv2.imread(i)
    i=np.array(x)
    if i.shape!=(256,256,3):
        break

image_dataset = []
for i, image_name in enumerate(images):
    img = Image.open(image_name).convert('L')
    x = asarray(img)
    image = cv2.resize(x,(256,256))
    image_dataset.append(image)

for i in range(len(image_dataset)):
    image_dataset[i]=image_dataset[i].reshape((256,256))

image_dataset = np.array(image_dataset)/255.
test_image_dataset=np.expand_dims((np.array(image_dataset)),3)

In [ ]:
model = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})
model1 = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})
model2 = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})
model3 = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})
#give models

In [ ]:
x=model4.predict(test_image_dataset)

In [ ]:
ans=[]
true=[]
from tqdm import tqdm
for index in tqdm(range(len(test_image_dataset[:700]))):
    test_img = test_image_dataset[index]
    test_img_input=np.expand_dims(test_img, 0)
    prediction = (model1.predict(test_img_input))
    predicted_img1=prediction
    predicted_img1[0][predicted_img1[0]<0.5] = 0
    predicted_img1[0][predicted_img1[0]>=0.5] = 1

    test_img = test_image_dataset[index]
    test_img_input=np.expand_dims(test_img, 0)
    prediction = (model2.predict(test_img_input))
    predicted_img2=prediction
    predicted_img2[0][predicted_img2[0]<0.5] = 0
    predicted_img2[0][predicted_img2[0]>=0.5] = 1

    test_img = test_image_dataset[index]
    test_img_input=np.expand_dims(test_img, 0)
    prediction = (model3.predict(test_img_input))
    predicted_img3=prediction
    predicted_img3[0][predicted_img3[0]<0.5] = 0
    predicted_img3[0][predicted_img3[0]>=0.5] = 1

    new = np.zeros((256,256))
    for i in range(256):
        for j in range(256):
            count_1=0
            count_0=0
            if predicted_img1[0][i][j]==1:
                count_1+=1
            else:
                count_0+=1
            if predicted_img2[0][i][j]==1:
                count_1+=1
            else:
                count_0+=1
            if predicted_img3[0][i][j]==1:
                count_1+=1
            else:
                count_0+=1

            if count_1>=count_0:
                new[i][j]=1
            else:
                new[i][j]=0

    ans.append(new)

true=np.array(true)
ans=np.array(ans)


In [ ]:
model1 = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})
model2 = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})
model3 = load_model('',custom_objects={'dice_coef_loss':dice_coef_loss,'dice_coef':dice_coef,'iou_score':iou_score})

In [ ]:
infection=[]
true=[]

from tqdm import tqdm
for index in tqdm(range(len(test_image_dataset[:700]))):

    test_img = test_image_dataset[index]
    test_img_input=np.expand_dims(test_img, 0)
    prediction = (model1.predict(test_img_input))
    predicted_img1=prediction
    predicted_img1[0][predicted_img1[0]<0.5] = 0
    predicted_img1[0][predicted_img1[0]>=0.5] = 1

    test_img = test_image_dataset[index]
    test_img_input=np.expand_dims(test_img, 0)
    prediction = (model2.predict(test_img_input))
    predicted_img2=prediction
    predicted_img2[0][predicted_img2[0]<0.5] = 0
    predicted_img2[0][predicted_img2[0]>=0.5] = 1

    test_img = test_image_dataset[index]
    test_img_input=np.expand_dims(test_img, 0)
    prediction = (model3.predict(test_img_input))
    predicted_img3=prediction
    predicted_img3[0][predicted_img3[0]<0.5] = 0
    predicted_img3[0][predicted_img3[0]>=0.5] = 1

    new = np.zeros((256,256))

    for i in range(256):
        for j in range(256):
            count_1=0
            count_0=0
            if predicted_img1[0][i][j]==1:
                count_1+=1
            else:
                count_0+=1
            if predicted_img2[0][i][j]==1:
                count_1+=1
            else:
                count_0+=1
            if predicted_img3[0][i][j]==1:
                count_1+=1
            else:
                count_0+=1

            if count_1>=count_0:
                new[i][j]=1
            else:
                new[i][j]=0

    infection.append(new)

true=np.array(true)
infection=np.array(infection)


In [ ]:
from tqdm import tqdm
for k in tqdm(range(len(ans))):
    lung=ans[k]
    inf=infection[k]
    counter_lung=0
    counter_inf=0
    for i in range(256):
        for j in range(256):
            if lung[i][j]==1:
                counter_lung+=1
            if inf[i][j]==1:
                counter_inf+=1
    div=(counter_inf/counter_lung)
    div=div*100
    if div>35 and div<50:
        print(k)
        break

In [ ]:
lung=ans[371]
inf=infection[371]
counter_lung=0
counter_inf=0
for i in range(256):
    for j in range(256):
        if lung[i][j]==1:
            counter_lung+=1
        if inf[i][j]==1:
            counter_inf+=1
div=(counter_inf/counter_lung)

In [ ]:
xx=np.expand_dims(infection[371],axis=-1)
ret,thresh=cv2.threshold(xx,127,255,0)
x=np.stack((ans[371],)*3,axis=-1)
image_filename = "ans_image.jpeg"
import matplotlib.image
matplotlib.image.imsave(image_filename, x)
img=cv2.imread("ans_image.jpeg")
img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
_,thresh=cv2.threshold(img_gray,127,255,cv2.THRESH_BINARY)
contours,hierarchy=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
contours = sorted(contours, key = cv2.contourArea)[-2:]
img=cv2.imread(images[371])
img = cv2.resize(img,(256,256))
image_filename = "test_image.jpeg"
matplotlib.image.imsave(image_filename, img)
x=np.stack((infection[371],)*3,axis=-1)
image_filename = "infection_image.jpeg"
matplotlib.image.imsave(image_filename, x)
img2=cv2.imread("infection_image.jpeg")
img_gray=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
_,thresh=cv2.threshold(img_gray,127,255,cv2.THRESH_BINARY)
contours,hierarchy=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
contours = sorted(contours, key = cv2.contourArea)[-2:]